# 1. Small-world network example

before you start read this short (2.5 pages) paper

"Emergence of Scaling in Random Networks", Albert Laszlo Barabasi and Reka Albert, https://web.archive.org/web/20120417112354/http://www.nd.edu/~networks/Publication%20Categories/03%20Journal%20Articles/Physics/EmergenceRandom_Science%20286,%20509-512%20(1999).pdf


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fname = Path('/home/anvar/projects/iitp_networks1/2019/assignments/itas_2009-2017_v8.xls')
itas_all = pd.read_excel(fname)

itas_all.head()

In [ ]:
title_author = itas_all.groupby(by=['title', 'author_id_new']).count().reset_index()
authors, titles = title_author.title, title_author.author_id_new
title_author.head(5)

In [ ]:
inc_table = pd.crosstab(authors, titles)
inc_table.head(5).iloc[:, :5]

In [ ]:
adjacency_matrix = inc_table.values.dot(inc_table.values.T)
n_papers = np.diag(adjacency_matrix)
np.fill_diagonal(adjacency_matrix, 0)

In [ ]:
adjacency_matrix[adjacency_matrix>1] = 1

In [ ]:
bins = np.histogram(adjacency_matrix.sum(axis=1), bins=20)

In [ ]:
plt.scatter(bins[1][1:], bins[0])
plt.plot(bins[1][1:], bins[0]);

In [ ]:
plt.scatter(np.log(bins[1][1:]), np.log(bins[0]))
plt.plot(np.log(bins[1][1:]), np.log(bins[0]));

### Fitting a line to estimate `alpha`

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:

model = LinearRegression()
model.fit(np.log(bins[1][1:]).reshape(-1,1), np.log(bins[0]))
x = np.linspace(1.2, 4.6, 100).reshape(-1,1)
y = model.predict(x)


plt.scatter(np.log(bins[1][1:]), np.log(bins[0]))
plt.plot(np.log(bins[1][1:]), np.log(bins[0]))

plt.plot(x, y, c='r', label=f'alpha = {np.round(model.coef_[0], 3)}')
plt.legend();

# Home reading

"Scale-free networks are rare", Anna D. Broido, Aaron Clauset https://www.nature.com/articles/s41467-019-08746-5

# 2. Build graphs in 3 popular libraries

# 2.1 `networkx`

In [ ]:
import networkx as nx

In [ ]:
edges = [
    (1, 2),
    (1, 3),
    (2, 3),
    (4, 5),
    (4, 6),
    (5, 6),
    (1, 4),
]
df_edges = pd.DataFrame(data=edges, columns=['source', 'target'])

In [ ]:
G = nx.from_pandas_edgelist(df_edges)

In [ ]:
# dir(G)

In [ ]:
adjacency = np.array(nx.adjacency_matrix(G).todense())
pos = nx.layout.fruchterman_reingold_layout(G)

In [ ]:
nx.draw(G, pos=pos)

# 2.2 `igraph`

In [ ]:
from igraph import Graph, ADJ_MAX

In [ ]:
plt.imshow(adjacency)
plt.colorbar();

In [ ]:
g0 = Graph.Weighted_Adjacency(
            adjacency.tolist(), mode=ADJ_MAX, attr='weight')

In [ ]:
# dir(g0)

In [ ]:
pos = g0.layout_fruchterman_reingold()
pos = dict(zip(range(1,7), pos.coords))

### `igraph` was originally designed for R language, so for plotting let's use `networkx` routines

In [ ]:
nx.draw(G, pos=pos)

# 2.3 `pytorch-geometric`

In [ ]:
import torch
from torch_geometric.data import Data
import torch_geometric

In [ ]:
# create edge index from nx graph

adj = nx.to_scipy_sparse_matrix(G).tocoo()
row = torch.from_numpy(adj.row.astype(np.int64)).to(torch.long)
col = torch.from_numpy(adj.col.astype(np.int64)).to(torch.long)
edge_index = torch.stack([row, col], dim=0)

In [ ]:
# pytorch-geometric allow for features both on vertices and on edges
x = torch.tensor([[-1], [0], [1], [1], [-1], [0]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

In [ ]:
# dir(data)

In [ ]:
g = torch_geometric.utils.to_networkx(data, to_undirected=True)
nx.draw(g)

# 3. Домашнее задание

1. Имплементировать модель Барабаси-Альберта: 

```
def generate_barabsi_albert(n: int, **args) -> np.ndarray:
    """Generates graph with `n` nodes via a preferential attachement process. 
    Returns its adjacency matrix."""
    ...
```

2. Визуализировать граф соавторств ИТИСа:
    - выделить гигантскую компоненту связности
    - визуализировать GCC используя любой метод укладки на выбор (методы igraph работают быстрее):
        - `dir(nx.layout)`
        - `[d for d in dir(igraph.Graph) if d.startswith('layout_')]`
    - на изображении подписать 10 вершин (авторов) с наибольшей степенью вершины
    
3. Сравнить граф соавторств ИТИСа и модельный граф Барабаси-Альберта (с таким же количеством вершин):
    - размер гигантской компоненты связности
    - распределение степеней вершин в log-log шкале
    - среднюю длину пути между вершинами внутри GCC
    - величину экспоненты $\alpha$: $k(x) \propto x^{\alpha}$
    
4.(*) можно ли модифицировать процесс preferential attachment чтобы генерить сеть с заданной наперед $\alpha$?